In [22]:
import torch

In [ ]:
#https://github.com/huggingface/transformers

In [23]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
    device = "cpu"
else:
    print('CUDA is available!  Training on GPU ...')
    device = torch.device("cuda:0")
    print("Using",torch.cuda.get_device_name(device))

CUDA is available!  Training on GPU ...
Using Tesla P100-PCIE-16GB


In [24]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 8.8 MB/s 
     |████████████████████████████████| 596 kB 60.9 MB/s 
     |████████████████████████████████| 6.6 MB 50.6 MB/s 
     |████████████████████████████████| 895 kB 56.4 MB/s 
     |████████████████████████████████| 77 kB 8.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [25]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [26]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import random
%matplotlib inline

In [27]:
import torch
#import torch.transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertConfig, DistilBertTokenizer

In [28]:
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset, random_split, DataLoader
import matplotlib.pyplot as plt
import torch.nn.functional as F
from collections import OrderedDict
from torchvision.utils import make_grid

In [29]:
data_dir = '/content/drive/MyDrive/study_project/spring/Irr&Re_Expanded_Manual.csv'

# Define train and test path 
#train_path = os.path.join(data_dir, 'train')
#test_path = 

In [30]:
import pandas as pd
book_dataset = pd.read_csv(data_dir)

In [31]:
book_dataset.head(5)

,topic_week,name_week,index,score,article_text,Re
0,10_11,10_11_adams_fein_ireland_clinton,29675,-0.186667,"By Christian ChaiseWASHINGTON, March 9 (AFP) -...",0
1,10_11,10_11_adams_fein_ireland_clinton,29390,-0.186667,President Bill Clinton cleared the way Thursda...,0
2,10_11,10_11_adams_fein_ireland_clinton,29772,-0.186667,London - The British press seethed yesterday w...,0
3,10_11,10_11_adams_fein_ireland_clinton,29928,-0.186667,It would be a mistake for Washington to lift a...,0
4,10_11,10_11_adams_fein_ireland_clinton,29663,-0.186667,British Prime Minister John Major will visit ...,0


In [32]:
book_dataset.info

<bound method DataFrame.info of      topic_week                         name_week  index     score  \
0         10_11  10_11_adams_fein_ireland_clinton  29675 -0.186667   
1         10_11  10_11_adams_fein_ireland_clinton  29390 -0.186667   
2         10_11  10_11_adams_fein_ireland_clinton  29772 -0.186667   
3         10_11  10_11_adams_fein_ireland_clinton  29928 -0.186667   
4         10_11  10_11_adams_fein_ireland_clinton  29663 -0.186667   
...         ...                               ...    ...       ...   
6525        9_9   9_9_fans_chelsea_bruges_belgian   6886 -0.197727   
6526        9_9   9_9_fans_chelsea_bruges_belgian   6463 -0.197727   
6527        9_9   9_9_fans_chelsea_bruges_belgian   6414 -0.197727   
6528        9_9   9_9_fans_chelsea_bruges_belgian   6807 -0.197727   
6529        9_9   9_9_fans_chelsea_bruges_belgian   7046 -0.197727   

                                           article_text  Re  
0     By Christian ChaiseWASHINGTON, March 9 (AFP) -...   0  
1  

In [33]:
# creating a copy of the articles
book_indx = book_dataset['index']

In [34]:
book_indx.count()

6530

In [35]:
book_df = book_dataset[['article_text','Re']]
book_df.head(3)

,article_text,Re
0,"By Christian ChaiseWASHINGTON, March 9 (AFP) -...",0
1,President Bill Clinton cleared the way Thursda...,0
2,London - The British press seethed yesterday w...,0


In [36]:
book_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6530 entries, 0 to 6529
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   article_text  6529 non-null   object
 1   Re            6530 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 102.2+ KB


In [37]:
book_df_na = book_df[book_df['article_text'].notna()]

In [38]:
book_df_na.count()

article_text    6529
Re              6529
dtype: int64

In [39]:
book_df_na.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6529 entries, 0 to 6529
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   article_text  6529 non-null   object
 1   Re            6529 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 153.0+ KB


dataloader

In [40]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')


Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [89]:
class Triage(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        title = str(self.data.article_text[index])
        title = " ".join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.Re[index], dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [91]:
print(train_dataset.head(3))


                                        article_text  Re
0  After a brief afternoon lull in fighting Wedne...   0
1  The story filtered around the office on recent...   1
2  RULE OF THE BONE, by Russell Banks. HarperColl...   1


In [62]:
# Creating the dataset and dataloader for the neural network

train_size = 0.8
train_dataset = book_df_na.sample(frac=train_size,random_state=200)
val_dataset = book_df_na.drop(train_dataset.index).reset_index(drop=True)

train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(book_df_na.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("VAL Dataset: {}".format(val_dataset.shape))

training_set = Triage(train_dataset, tokenizer, MAX_LEN)
val_set = Triage(val_dataset, tokenizer, MAX_LEN)

FULL Dataset: (6529, 2)
TRAIN Dataset: (5223, 2)
VAL Dataset: (1306, 2)


In [64]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

val_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
val_loader = DataLoader(val_set, **val_params)

Finetuning

In [65]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class DistillBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistillBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [66]:
model = DistillBERTClass()
model.to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DistillBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_feat

In [67]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [68]:
# Function to calcuate the accuracy of the model

def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [69]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accu(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [70]:
for epoch in range(EPOCHS):
    train(epoch)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2272: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Training Loss per 5000 steps: 0.6191117763519287
Training Accuracy per 5000 steps: 100.0
The Total Accuracy for Epoch 0: 76.35458548726785
Training Loss Epoch: 0.5014578501221825
Training Accuracy Epoch: 76.35458548726785


In [ ]:
# for epoch in range(EPOCHS):
#     train(epoch)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Training Loss per 5000 steps: 1.4445347785949707
Training Accuracy per 5000 steps: 0.0
The Total Accuracy for Epoch 0: 74.9186291403408
Training Loss Epoch: 0.543683642550025
Training Accuracy Epoch: 74.9186291403408


In [72]:
def valid(model, testing_loader):
    tr_loss = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accu(big_idx, targets)
            #print(n_correct)
            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
    print("Got {}/ {} correct on Val set". format(n_correct,nb_tr_examples))
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    return epoch_accu

In [73]:
print('This is the validation section to print the accuracy and see how it performs')
print('Here we are leveraging on the dataloader crearted for the validation dataset, the approcah is using more of pytorch')

val_acc = valid(model, val_loader)
print("Accuracy on Val data = %0.2f%%" % val_acc)

This is the validation section to print the accuracy and see how it performs
Here we are leveraging on the dataloader crearted for the validation dataset, the approcah is using more of pytorch
Validation Loss per 100 steps: 0.1598212867975235
Validation Accuracy per 100 steps: 100.0


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2272: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Got 1070/ 1306 correct on Val set
Validation Loss Epoch: 0.40925349487964446
Validation Accuracy Epoch: 81.92955589586524
Accuracy on Val data = 81.93%


In [ ]:
# print('This is the validation section to print the accuracy and see how it performs')
# print('Here we are leveraging on the dataloader crearted for the validation dataset, the approcah is using more of pytorch')

# acc = valid(model, testing_loader)
# print("Accuracy on test data = %0.2f%%" % acc)

This is the validation section to print the accuracy and see how it performs
Here we are leveraging on the dataloader crearted for the validation dataset, the approcah is using more of pytorch
Validation Loss per 100 steps: 0.8423527479171753
Validation Accuracy per 100 steps: 50.0


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Validation Loss Epoch: 0.3954950426783624
Validation Accuracy Epoch: 83.2312404287902
Accuracy on test data = 83.23%


Saving model

In [75]:
# Saving the files for re-use

output_model_file = 'pytorch_distilbert_news_NEW.bin'
output_vocab_file = 'vocab_distilbert_news_NEW.bin'


In [76]:

model_to_save = model
torch.save(model_to_save, output_model_file)
#tokenizer.save_vocabulary(output_vocab_file)

print('Files saved')


Files saved


In [ ]:
# weights_fname = 'xyz.pth'
# torch.save(model.state_dict(), weights_fname)

In [77]:
for i in val_loader:
  print(i)
  break

{'ids': tensor([[  101, 17173, 22089,  ...,     0,     0,     0],
        [  101,  7239,   118,  ...,     0,     0,     0]]), 'mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'targets': tensor([0, 0])}


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2272: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Loading trained model

In [78]:
model_path = "/content/drive/MyDrive/study_project/spring/pytorch_distilbert_news_NEW.bin"

In [79]:
trained_model = torch.load(model_path)

In [80]:
trained_model

DistillBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_feat

Testing set 


In [ ]:
# # Example for a Bert model
# config = BertConfig.from_json_file(output_config_file)
# model = BertForQuestionAnswering(config)
# state_dict = torch.load(output_model_file)
# model.load_state_dict(state_dict)
# tokenizer = BertTokenizer(output_vocab_file, do_lower_case=args.do_lower_case)

In [81]:
test_datadir = '/content/drive/MyDrive/study_project/spring/irrelevant_relevant_full_1995.csv'

# Define train and test path 
#train_path = os.path.join(data_dir, 'train')
#test_path = 

In [83]:
import pandas as pd
#local_file="all_emails.csv"
test_dataset = pd.read_csv(test_datadir)

In [84]:
test_dataset.head(5)

,Unnamed: 0,rn,article_text,Re
0,0,4753991,One of the most famous photographic works of t...,1.0
1,1,827021,After a bumpy year of bankruptcy and bidding w...,1.0
2,2,340448,"A funeral service for Elwin J. Carpenter Sr., ...",1.0
3,3,5334687,"James Rodgers Maj Earl, 77, a retired Army maj...",1.0
4,4,6107329,Christmas in Colonial Williamsburg is like no ...,1.0


In [85]:
test_dataset.info

<bound method DataFrame.info of        Unnamed: 0       rn                                       article_text  \
0               0  4753991  One of the most famous photographic works of t...   
1               1   827021  After a bumpy year of bankruptcy and bidding w...   
2               2   340448  A funeral service for Elwin J. Carpenter Sr., ...   
3               3  5334687  James Rodgers Maj Earl, 77, a retired Army maj...   
4               4  6107329  Christmas in Colonial Williamsburg is like no ...   
...           ...      ...                                                ...   
43767       43767  5040528  John Berendt put Savannah, Georgia and himself...   
43768       43768  4457494  Seinfeld (PG), Ten at 7.30pm. Jerry faces a qu...   
43769       43769  3113796  'NOTHING is serious. The flippant remark by Tr...   
43770       43770  1770306  PERMANENTLY glum-faced and highly strung worke...   
43771       43771  1858262  Author Stephen Beaumont, whose A Taste For Bee...

In [86]:
test_df= test_dataset[['article_text','Re']]
test_df.head(3)

,article_text,Re
0,One of the most famous photographic works of t...,1.0
1,After a bumpy year of bankruptcy and bidding w...,1.0
2,"A funeral service for Elwin J. Carpenter Sr., ...",1.0


In [87]:
test_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43772 entries, 0 to 43771
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   article_text  43772 non-null  object 
 1   Re            43772 non-null  float64
dtypes: float64(1), object(1)
memory usage: 684.1+ KB


In [94]:
# Creating the dataset and dataloader for the neural network

# training_set = Triage(train_dataset, tokenizer, MAX_LEN)
# val_set = Triage(val_dataset, tokenizer, MAX_LEN)
test_set = Triage(test_df, tokenizer, MAX_LEN)
test_set


In [95]:
test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }


In [ ]:
# train_params = {'batch_size': TRAIN_BATCH_SIZE,
#                 'shuffle': True,
#                 'num_workers': 0
#                 }

# val_params = {'batch_size': VALID_BATCH_SIZE,
#                 'shuffle': True,
#                 'num_workers': 0
#                 }

# training_loader = DataLoader(training_set, **train_params)
# val_loader = DataLoader(val_set, **val_params)

In [96]:
test_loader = DataLoader(test_set, **test_params)

In [106]:
def test(model, testing_loader):
    #tr_loss = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask).squeeze()
            #loss = loss_function(outputs, targets)
            #tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accu(big_idx, targets)
            #print(n_correct)
            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%5000==0:
                #loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                #print(f"Testing Loss per 100 steps: {loss_step}")
                print(f"Testing Accuracy per 100 steps: {accu_step}")
    print("Got {}/ {} correct on Test set". format(n_correct,nb_tr_examples))
    #epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    #print(f" Loss Epoch: {epoch_loss}")
    print(f"Testing Accuracy Epoch: {epoch_accu}")
    
    return epoch_accu

In [107]:
test_acc = test(trained_model,test_loader)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2272: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Testing Accuracy per 100 steps: 50.0
Testing Accuracy per 100 steps: 82.95340931813637
Testing Accuracy per 100 steps: 83.29167083291671
Testing Accuracy per 100 steps: 83.15778948070128
Testing Accuracy per 100 steps: 82.96335183240838
Got 36333/ 43772 correct on Test set
Testing Accuracy Epoch: 83.00511742666545


In [109]:
print("Accuracy on Test data = %0.2f%%" % test_acc)

Accuracy on Val data = 83.01%
